In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import numpy as np

In [ ]:
class CelsiusFahrenheitModel(pl.LightningModule):

    def __init__(self, batch_size=32):
        super(CelsiusFahrenheitModel, self).__init__()
        self.model = self.get_model()
        self.criterion = self.get_criterion()

        self.batch_size = batch_size

    def get_model(self): 
        return nn.Sequential(nn.Linear(1, 1))

    def get_criterion(self):
        return nn.MSELoss()    

    def get_optimizer(self):
        return optim.Adam(self.model.parameters(), lr=0.1)  

    def forward(self, x):
        # called with self(x)
        return self.model(x)

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        return {'loss': loss}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'val_loss': avg_loss}

    def configure_optimizers(self):
        # REQUIRED
        return self.get_optimizer()

    def prepare_data(self) -> None:
        X = np.arange(-100, 100, 0.1)
        y = np.array([x*9/5+32 for x in X])

        X_train = torch.from_numpy(X.reshape(-1,1)).float()
        y_train = torch.from_numpy(y.reshape(-1,1)).float()
        trainset = torch.utils.data.TensorDataset(X_train, y_train)
        valsize = int(len(trainset)*.2)
        self.trainset, self.valset = torch.utils.data.random_split(trainset, [len(trainset)-valsize, valsize])

    def train_dataloader(self):
        # REQUIRED
        return torch.utils.data.DataLoader(self.trainset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # OPTIONAL
        return torch.utils.data.DataLoader(self.valset, batch_size=self.batch_size)

In [ ]:
num_epochs = 20

cf_model = CelsiusFahrenheitModel()

trainer = pl.Trainer(gpus=1, early_stop_callback=True, max_nb_epochs=num_epochs)    
trainer.fit(cf_model)
trainer.callback_metrics

In [ ]:
for name, param in cf_model.model.named_parameters():
    print(name, param)